In [20]:
import requests
import pandas as pd
import math
import time
import bs4
import lxml
import re
import html as ihtml
from bs4 import BeautifulSoup

In [21]:
api_token = '2gaZEtABi5pxX1tFQeRgudXuWheouJKgYemjwUO58EH15WEKQJGw4NRXeDpbyH+r91fmkmv2ycO9uRH16yBIsQ=='    
user_id = 'cnzHhxx5yqwWiUl'

#Dataframe for saving job IDs
jobs_df = pd.DataFrame(columns = ["ID", "Title", "Company", "Location"])

In [22]:
#CareerOneStop/NLX
#First step is to gather job list; each job IDs will then be queried for specific job details/descriptions in the second step
#The maximum number of results is 500; script must iterate through pages to aggregate all the job IDs.

query_term = 'cybersecurity'
days = '0' #All days
start_page = 1
end_page = 500
url = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{query_term}/United%20States/25/0/0/{start_page}/{end_page}/{days}?showFilters=false'
headers = {'Authorization': 'Bearer ' + api_token,
 'Content-Type': 'application/json'}
results = requests.get(url, headers=headers).json()



In [23]:
#Calculate number of API calls to get all the results
jobs_total = int(results['Jobcount'])
total_pages = math.ceil(int(results['Jobcount'])/500)
print('Number of total jobs: ' + str(jobs_total) , '\nNumber of iterations/pages: ' + str(total_pages))

Number of total jobs: 52566 
Number of iterations/pages: 106


In [27]:
#Loop through all pages of jobs
#Adjust query term and new results will be added to dataframe

query_term = 'cybersecurity'
for i in range(total_pages):

    url = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{query_term}/United%20States/25/0/0/{start_page}/{end_page}/{days}?showFilters=false'
    #Extract job IDs and append to master list
    results = requests.get(url, headers=headers).json()
    for i, value in enumerate(results['Jobs']):
        jobs_df = jobs_df.append(pd.DataFrame([[results['Jobs'][i]['JvId'],results['Jobs'][i]['JobTitle'],results['Jobs'][i]['Company'],results['Jobs'][i]['Location']]], columns = ["ID", "Title", "Company", "Location"] ))
    start_page += 500
    end_page += 500
    time.sleep(1.0)
        

In [28]:
#Simple filter on job titles
cyber_jobs = jobs_df[jobs_df['Title'].str.contains("cyber",case = False)].reset_index(drop = True)

In [29]:
cyber_jobs

,ID,Title,Company,Location
0,7C2321CECB83415B9D1D9854F8867997206,Cybersecurity,US Navy,"Colorado Springs, Colorado"
1,F4B11A16CE5740A898C929513BA7AFE5206,Cybersecurity,US Navy,"Detroit, Michigan"
2,89A5F5B490BA43EF816D7192BB369A1A206,Cybersecurity,Idaho Department of Labor,"Nampa, Idaho"
3,6640417C13B24BBDBEDA468648E26A8C206,Cybersecurity,Arizona Department of Economic Security,"Phoenix, Arizona"
4,A8876D50F0784C15860AD9099F277F1F206,principal - cybersecurity,Randstad US,"frisco, Texas"
...,...,...,...,...
12242,69968E703B1C4FC98D68232D448AEA02206,"Senior Technical Security Analyst, Cyber Risk ...",Splunk,"San Jose, California"
12243,7EAFF35B3E3241B2841D7F412ED6E0A1206,Cyber Security Analyst,ISC Consulting Group,"Augusta, Georgia"
12244,4727F27733084669AE4F359859D13347206,Lead Engineer – Cyberphysical Cloud Operations,National Renewable Energy Laboratory,"Golden, Colorado"
12245,FBA276E4970F4CAE8E81A415A8E5B331206,Senior Cyber Security Analyst - Standardizatio...,UIC Government Services and the Bowhead Family...,"San Antonio, Texas"


In [128]:
#Retrieve job details using list of job IDs
###
#This will generate large number of calls to API!
###
jobs = pd.DataFrame()
for i, value in enumerate(cyber_jobs['ID']):
    job_id = cyber_jobs.iloc[i]['ID']
    try:
        url_getjob = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{job_id}?isHtml=true&enableMetaData=true'
        results = requests.get(url_getjob, headers=headers)
        if results.status_code == 200:
            results = requests.get(url_getjob, headers=headers).json()
            pd.DataFrame([[cyber_jobs.iloc[i]['ID'],results['JobTitle'], results['Company'], results['Description']]]).to_csv('Jobs2.csv', mode='a', index=False, header=False)
            time.sleep(1.5) #throttle the calls down to once per second!
        elif results[0]['Description'] == 'Job No Longer Available':
            continue
    except Exception as inst:
        print(inst)
        #time.sleep(60)
        continue

'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not 

In [12]:
df = pd.read_csv('C:\\Users\\Joe\\Documents\\GitHub\\CyberWorkForceProject\\Data\\Cleaned Data\\NLX Jobs.csv')

def clean_text(text):
    text = BeautifulSoup(ihtml.unescape(text), "lxml").text
    text = re.sub(r"\s+", " ", text)
    return text
df['Description'] = df['Description'].apply(clean_text)

In [63]:
#Merge dfs to add location column to final dataset
test = df.merge(cyber_jobs, on ="ID",how='left')

In [67]:
test.to_csv('C:\\Users\\Joe\\Documents\\GitHub\\CyberWorkForceProject\\Data\\Cleaned Data\\NLX Jobstest.csv')